In [47]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold



#評価用

train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

with open('ensamble/rf_train.pickle', mode='br') as fi:
    train_p1 = pickle.load(fi)
    
with open('ensamble/lgb_train.pickle', mode='br') as fi:
    train_p2 = pickle.load(fi)
    
with open('ensamble/tabnet_train.pickle', mode='br') as fi:
    train_p3 = pickle.load(fi)
    
with open('ensamble/nn_train.pickle', mode='br') as fi:
    train_p4 = pickle.load(fi)
    
with open('ensamble/lgr_train.pickle', mode='br') as fi:
    train_p5 = pickle.load(fi)






#データ読み込み
with open('ensamble/rf_test.pickle', mode='br') as fi:
    test_p1 = pickle.load(fi)
    
with open('ensamble/lgb_test.pickle', mode='br') as fi:
    test_p2 = pickle.load(fi)
    
with open('ensamble/tabnet_test.pickle', mode='br') as fi:
    test_p3 = pickle.load(fi)
    
with open('ensamble/nn_test.pickle', mode='br') as fi:
    test_p4 = pickle.load(fi)
    
with open('ensamble/lgr_test.pickle', mode='br') as fi:
    test_p5 = pickle.load(fi)
    

## アンサンブル

In [65]:
#　評価用
df_train = pd.concat(
    [train['Outcome'],train_p1['pred'],train_p2['pred'], train_p3['pred'], train_p4['pred'], train_p5['pred']], 
                     axis=1, ignore_index=True).rename(columns={
    0: 'true', 1: 'pred1', 2: 'pred2', 3:'pred3', 4:'pred4',5:'pred5'
})

df_train['ensemble'] = df_train['pred1']*0.3 + df_train['pred2']*0.2 +df_train['pred3']*0.3 + df_train['pred4']*0.1 +  df_train['pred5']*0.1

# 提出用
df_test = pd.concat(
    [test['index'],test_p1['pred'],test_p2['pred'], test_p3['pred'],  test_p4['pred'], test_p5['pred']], 
                    axis=1, ignore_index=True).rename(columns={
    0: 'index', 1: 'pred1', 2: 'pred2', 3:'pred3', 4:'pred4', 5:'pred5'
})

df_test['ensemble'] = df_test['pred1']*0.3 + df_test['pred2']*0.2 + df_test['pred3']*0.3 + df_test['pred4']*0.1 + df_test['pred5']*0.1


def evaluate_ensemble(input_df, col_pred):
    print('[auc] model1:{:.4f},model2:{:.4f},model3:{:.4f},model4:{:.4f},model5:{:.4f}  ->    emsamble]{:.4f}'.format(
        roc_auc_score(input_df['true'], input_df['pred1']),
        roc_auc_score(input_df['true'], input_df['pred2']),
        roc_auc_score(input_df['true'], input_df['pred3']),
        roc_auc_score(input_df['true'], input_df['pred4']),
        roc_auc_score(input_df['true'], input_df['pred5']),
        
        
        roc_auc_score(input_df['true'], input_df[col_pred]),
    ))

In [66]:
df_train.head(20)

,true,pred1,pred2,pred3,pred4,pred5,ensemble
0,1.0,0.463801,0.628059,0.560029,0.384083,0.402233,0.511392
1,0.0,0.083710,0.053503,0.061435,0.104301,0.073798,0.072054
2,0.0,0.497738,0.371698,0.655945,0.444559,0.412125,0.506113
3,0.0,0.638009,0.586979,0.676285,0.486867,0.602159,0.620587
4,1.0,0.800905,0.561965,0.972826,0.790010,0.689416,0.792455
5,0.0,0.061086,0.119255,0.025724,0.124213,0.173991,0.079714
6,1.0,0.090498,0.051957,0.046483,0.131972,0.124462,0.077129
7,1.0,0.518100,0.373684,0.440282,0.459732,0.519163,0.460141
8,0.0,0.185520,0.118094,0.202881,0.167233,0.202901,0.177153
9,0.0,0.352941,0.288413,0.493380,0.382196,0.347957,0.384594


In [67]:
evaluate_ensemble(df_train, col_pred='ensemble')

[auc] model1:0.8053,model2:0.8182,model3:0.7668,model4:0.7573,model5:0.7537  ->    emsamble]0.8107


In [68]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('sub/submission_rf+lgb+tab.csv', index=None, header=False,)

## スタッキング

In [69]:
x, y = df_train[['pred1', 'pred2','pred3','pred4', 'pred5']], df_train[['true']]
oof = np.zeros(len(x))
models = []

cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x,y))
for nfold in np.arange(5):
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x.loc[idx_tr, :], y.loc[idx_tr, :]
    x_va, y_va = x.loc[idx_va, :], y.loc[idx_tr, :]
    
    model = Lasso(alpha=0.01)
    model.fit(x_tr, y_tr)
    models.append(model)
    
    y_va_pred = model.predict(x_va)
    oof[idx_va] = y_va_pred
    
df_train['stacking'] = oof
df_train['stacking'] = df_train['stacking'].clip(lower=0, upper=1)
df_train.head(10)

,true,pred1,pred2,pred3,pred4,pred5,ensemble,stacking
0,1.0,0.463801,0.628059,0.560029,0.384083,0.402233,0.511392,0.527178
1,0.0,0.083710,0.053503,0.061435,0.104301,0.073798,0.072054,0.090633
2,0.0,0.497738,0.371698,0.655945,0.444559,0.412125,0.506113,0.369170
3,0.0,0.638009,0.586979,0.676285,0.486867,0.602159,0.620587,0.522481
4,1.0,0.800905,0.561965,0.972826,0.790010,0.689416,0.792455,0.576198
5,0.0,0.061086,0.119255,0.025724,0.124213,0.173991,0.079714,0.136804
6,1.0,0.090498,0.051957,0.046483,0.131972,0.124462,0.077129,0.090138
7,1.0,0.518100,0.373684,0.440282,0.459732,0.519163,0.460141,0.376657
8,0.0,0.185520,0.118094,0.202881,0.167233,0.202901,0.177153,0.155453
9,0.0,0.352941,0.288413,0.493380,0.382196,0.347957,0.384594,0.290308


In [70]:
evaluate_ensemble(df_train, col_pred='stacking')

[auc] model1:0.8053,model2:0.8182,model3:0.7668,model4:0.7573,model5:0.7537  ->    emsamble]0.8190


In [71]:
sub = df_test.copy()    
sub['ensemble'] = np.where(sub['ensemble'] < 0.5, 0, 1)
sub = sub[['index', 'ensemble']] 
sub.to_csv('sub/submission_stack.csv', index=None, header=False,)